In [2]:
import json
import numpy as np

In [3]:
import torch
print(torch.__version__)


2.10.0


In [5]:
def prepare_TStroop(rec, features, TStroop_way = 1):
    if TStroop_way == 1:
        seq_all = []  # 4 подтеста
        for key in ["mono", "trueColor", "color", "trueText"]:
            seq = []
            for row in rec["test_results"][key]:
                values = []
                for f in features:
                    values.append(row[f])  # добавляем в values данные по признакам
                seq.append(values)  # добавляем в seq строку
            seq_all.append(np.array(seq, dtype=float))  # каждый подтест отдельный массив
        return np.array(seq_all), rec["age"]  # массив из 4 таблиц, размерность 4


    if TStroop_way == 2:
        seq =[]
        for key in ["mono", "trueColor", "color", "trueText"]: #всё в одной таблице
            for row in rec["test_results"][key]:
                values = []
                for f in features:    
                    values.append(row[f])    #добавляем в values данные по нужным признакам (строка)
                seq.append(values)       #добавляем в seq строку
    return np.array(seq, dtype=float), rec["age"]

def prepare_test(rec, features):
    seq = []
    for row in rec["test_results"]: #список словарей
        values = []
        for f in features:
            values.append(row[f])    #добавляем в values данные по нужным признакам (строка)
        seq.append(values)       #добавляем в seq строку
    return np.array(seq, dtype=float), rec["age"]

def prepare_data(data, test_name: str, features: list[str], TStroop_way = 1):  #TStroop_way = 1 если разделяем mono и trueColor, 2 если нет

    X_list = []
    y_list = []
    for rec in data:
        if rec["test_name"] == "TStroop" and test_name == "TStroop":
            X_seq, y_val = prepare_TStroop(rec, features, TStroop_way)
            X_list.append(X_seq)  # добавляем либо 3д либо 4д массив
            y_list.append(y_val)  

        elif rec["test_name"] == test_name:
                X_seq, y_val = prepare_test(rec, features)
                X_list.append(X_seq) #добавляем таблицу по одной попытке теста
                y_list.append(y_val) 

    if test_name == "TStroop" and TStroop_way == 1:
        # Обработка для TStroop с разделением на подтесты
        num_subtests = 4  # фиксированное количество подтестов
        max_len = max(seq.shape[1] for seq in X_list)  # максимальная длина подтеста
        num_features = X_list[0].shape[2]
        batch_size = len(X_list)
        
        X = np.zeros((batch_size, num_subtests, max_len, num_features), dtype=float)#  Padding (выравнивание по длине) 
        for i, seq in enumerate(X_list):
            X[i, :, :seq.shape[1], :] = seq#заполняем тензор данными, там где их нет, оставляем 0
    else:
        # Обработка для остальных случаев
        max_len = max(seq.shape[0] for seq in X_list)
        num_features = X_list[0].shape[-1]  # берем последнюю размерность
        batch_size = len(X_list)
        
        X = np.zeros((batch_size, max_len, num_features), dtype=float)
        for i, seq in enumerate(X_list):
            X[i, :seq.shape[0], :] = seq

    y = np.array(y_list, dtype=float)
    return X, y


(batch_size, num_subtests, max_len, num_features) или (batch_size, max_len, num_features)

In [6]:
json_path = "calculator_9_2025-03-19_filter.json"
test_name = ["T1back", "TStroop", "T258", "T274", "T278"]
features_T1back = [
    "Stimul", "Goal", "Color", "Duration",
    "Show", "Hide", "Down", "Up",
    "SMR", "MR", "ERR_1", "ERR_2", "ERR_3"
]
features_TStroop = ["True", "Error", "SMR", "MR", "Time"]
features_T258 = ["Stimul", "H", "H+", "dH+", "H-", "dH-", "t+", "t-", "ERR", "ERR_LIM"]
features_T274 = ["Stimul", "Goal", "Duration", "Interval", "Show", "Hide", "Down", "Up", "SMR", "MR", "ERR_1", "ERR_2", "ERR_3"]
features_T278 = features_T274

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

x, y = prepare_data(data, test_name[1], features_TStroop, 1)    
print(f" Размерность X: {x.shape}, размерность Y: {y.shape}")

 Размерность X: (5040, 4, 29, 5), размерность Y: (5040,)


In [7]:
np.save("X.npy", x) # сохранение в файл
np.save("y.npy", y)



In [ ]:
t1back_data = [row for row in data if row["test_name"] == "T1back"] #список словарей

print(f"Найдено {len(t1back_data)} записей для T1back")
t1back_data[1]

Найдено 5040 записей для T1back


{'test_results': [{'#': 0,
   'Stimul': 2,
   'Goal': 0,
   'Color': 5,
   'ColorHTML': '#804040',
   'Place': 1,
   'Size': 1,
   'Duration': 300,
   'Interval': 2000,
   'Show': 2003,
   'Hide': 2316,
   'Down': None,
   'Up': None,
   'SMR': None,
   'MR': None,
   'ERR_1': 0,
   'ERR_2': 0,
   'ERR_3': 0},
  {'#': 1,
   'Stimul': 5,
   'Goal': 0,
   'Color': 2,
   'ColorHTML': '#00ff00',
   'Place': 1,
   'Size': 1,
   'Duration': 300,
   'Interval': 2000,
   'Show': 4016,
   'Hide': 4318,
   'Down': None,
   'Up': None,
   'SMR': None,
   'MR': None,
   'ERR_1': 0,
   'ERR_2': 0,
   'ERR_3': 0},
  {'#': 2,
   'Stimul': 5,
   'Goal': 0,
   'Color': 3,
   'ColorHTML': '#ffff00',
   'Place': 1,
   'Size': 1,
   'Duration': 300,
   'Interval': 2000,
   'Show': 6018,
   'Hide': 6330,
   'Down': None,
   'Up': None,
   'SMR': None,
   'MR': None,
   'ERR_1': 0,
   'ERR_2': 0,
   'ERR_3': 0},
  {'#': 3,
   'Stimul': 2,
   'Goal': 1,
   'Color': 3,
   'ColorHTML': '#ffff00',
   'Place': 1

In [ ]:
test_name = set()
tstroop_data = [row for row in data if row["test_name"] == "TStroop"]
for row in tstroop_data:
    for podtest in row["test_results"].keys():
        test_name.add(podtest)

test_name

{'color', 'mono', 'trueColor', 'trueText'}

Подсчет длины тестов

In [7]:
# Для TStroop теста
sample_tstroop = [rec for rec in data if rec["test_name"] == "TStroop"][0]
for subtest, values in sample_tstroop["test_results"].items():
    print(f"{subtest}: {len(values)}")

# Для других тестов
for test_name in ["T1back", "T258", "T274", "T278"]:
    sample_test = [rec for rec in data if rec["test_name"] == test_name][0]
    print(f"{test_name}: {len(sample_test['test_results'])}")

mono: 29
color: 29
trueText: 29
trueColor: 29
T1back: 38
T258: 13
T274: 15
T278: 30
